### prepare

In [151]:
import numpy as np
import pandas as pd
import cv2
from glob import glob
import spacy
import re
import string
import matplotlib.pyplot as plt
import easyocr
# import pytesseract

In [152]:
def cleanText(txt):
    whitespace = string.whitespace
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)

    return str(removepunctuation)

In [153]:
import warnings
warnings.filterwarnings('ignore')

In [154]:
### Load NER model
model_ner = spacy.load('output/model-best_v1/')

In [155]:
reader = easyocr.Reader(['th','en']) # this needs to run only once to load the model into memory

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [156]:
# Load Image
# image = cv2.imread('./data/test3.jpg')
# image = cv2.imread('../img_test/test10.jpg')
image = cv2.imread('./dataset/print_2.jpg')

# cv2.imshow('businesscard',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# tessData = pytesseract.image_to_data(image, lang='tha+eng', config='--psm 6')
# tessList = list(map(lambda x: x.split('\t'), tessData.split('\n')))
# df = pd.DataFrame(tessList[1:], columns=tessList[0])
# extract data using Pytesseract
tessData = reader.readtext(image, detail = 0)
# convert into dataframe
tessList = {'text' : tessData}
df = pd.DataFrame(tessList)
df.dropna(inplace=True) # drop missing values
df['text'] = df['text'].apply(cleanText)

# convet data into content
df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']])
print(content)
# get prediction from NER model
doc = model_ner(content)

ทรe manpuku manpuku โทรคพ10624344538 ปล ทานฟีร้น โต๊ะ-15 ช้อพนักงาน คุณปรม เวลาเป้า 19-04-25661638 เวลาท์ฟิมฟ์ 19-04-25661707 สินค้า oty ราคา ราคารวม 1.ทงคดฝึราเมนxl 1 139.00 139,00 2.ฟันตันเมนxl 2 149.00 298.00 3 ชาจีนrefill 3 20,00 60.00 เdน 4. ชาจีนrefill 20,00 20,00 ร้อน มอดรวม 517.00 ลด55 25.85 ฟั้งหมด 3491.15 haveanicemeal poweredbyocha


In [157]:
print(doc)

ทรe manpuku manpuku โทรคพ10624344538 ปล ทานฟีร้น โต๊ะ-15 ช้อพนักงาน คุณปรม เวลาเป้า 19-04-25661638 เวลาท์ฟิมฟ์ 19-04-25661707 สินค้า oty ราคา ราคารวม 1.ทงคดฝึราเมนxl 1 139.00 139,00 2.ฟันตันเมนxl 2 149.00 298.00 3 ชาจีนrefill 3 20,00 60.00 เdน 4. ชาจีนrefill 20,00 20,00 ร้อน มอดรวม 517.00 ลด55 25.85 ฟั้งหมด 3491.15 haveanicemeal poweredbyocha


In [158]:
from spacy import displacy

In [159]:
# displacy.serve(doc,style='ent')

In [160]:
displacy.render(doc,style='ent')

### Tagging

In [161]:
docjson = doc.to_json()
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [162]:
doc.to_json()

{'text': 'ทรe manpuku manpuku โทรคพ10624344538 ปล ทานฟีร้น โต๊ะ-15 ช้อพนักงาน คุณปรม เวลาเป้า 19-04-25661638 เวลาท์ฟิมฟ์ 19-04-25661707 สินค้า oty ราคา ราคารวม 1.ทงคดฝึราเมนxl 1 139.00 139,00 2.ฟันตันเมนxl 2 149.00 298.00 3 ชาจีนrefill 3 20,00 60.00 เdน 4. ชาจีนrefill 20,00 20,00 ร้อน มอดรวม 517.00 ลด55 25.85 ฟั้งหมด 3491.15 haveanicemeal poweredbyocha',
 'ents': [{'start': 49, 'end': 56, 'label': 'B-ITEM'},
  {'start': 150, 'end': 165, 'label': 'B-ITEM'},
  {'start': 175, 'end': 181, 'label': 'B-COST'},
  {'start': 182, 'end': 195, 'label': 'B-ITEM'},
  {'start': 205, 'end': 211, 'label': 'B-COST'},
  {'start': 214, 'end': 225, 'label': 'B-ITEM'},
  {'start': 234, 'end': 239, 'label': 'B-TOTAL'},
  {'start': 309, 'end': 316, 'label': 'B-TOTAL'}],
 'tokens': [{'id': 0, 'start': 0, 'end': 3},
  {'id': 1, 'start': 4, 'end': 11},
  {'id': 2, 'start': 12, 'end': 19},
  {'id': 3, 'start': 20, 'end': 36},
  {'id': 4, 'start': 37, 'end': 39},
  {'id': 5, 'start': 40, 'end': 48},
  {'id': 6, '

In [163]:
doc_text = docjson['text']

In [164]:
datafram_tokens = pd.DataFrame(docjson['tokens'])
datafram_tokens['token'] = datafram_tokens[['start','end']].apply(
    lambda x:doc_text[x[0]:x[1]] , axis = 1)
datafram_tokens.head(10)

,id,start,end,token
0,0,0,3,ทรe
1,1,4,11,manpuku
2,2,12,19,manpuku
3,3,20,36,โทรคพ10624344538
4,4,37,39,ปล
5,5,40,48,ทานฟีร้น
6,6,49,56,โต๊ะ-15
7,7,57,67,ช้อพนักงาน
8,8,68,74,คุณปรม
9,9,75,83,เวลาเป้า


In [165]:
right_table = pd.DataFrame(docjson['ents'])[['start','label']]
datafram_tokens = pd.merge(datafram_tokens,right_table,how='left',on='start')

In [166]:
datafram_tokens.fillna('O',inplace=True)
datafram_tokens.head(10)

,id,start,end,token,label
0,0,0,3,ทรe,O
1,1,4,11,manpuku,O
2,2,12,19,manpuku,O
3,3,20,36,โทรคพ10624344538,O
4,4,37,39,ปล,O
5,5,40,48,ทานฟีร้น,O
6,6,49,56,โต๊ะ-15,B-ITEM
7,7,57,67,ช้อพนักงาน,O
8,8,68,74,คุณปรม,O
9,9,75,83,เวลาเป้า,O


In [167]:
# join lable to df_clean dataframe
df_clean['end'] = df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1
df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)

In [168]:
datafram_tokens.tail(10)

,id,start,end,token,label
43,43,265,270,"20,00",O
44,44,271,275,ร้อน,O
45,45,276,282,มอดรวม,O
46,46,283,289,517.00,O
47,47,290,294,ลด55,O
48,48,295,300,25.85,O
49,49,301,308,ฟั้งหมด,O
50,50,309,316,3491.15,B-TOTAL
51,51,317,330,haveanicemeal,O
52,52,331,344,poweredbyocha,O


### start my way

In [169]:
datafram_tokens = datafram_tokens.query('token not in [".",","," "]')
datafram_tokens['token_shift_-1'] = datafram_tokens['token'].shift(-1)
datafram_tokens['token_shift_-2'] = datafram_tokens['token'].shift(-2)
datafram_tokens['token_shift_1'] = datafram_tokens['token'].shift(1)
datafram_tokens['token_shift_2'] = datafram_tokens['token'].shift(2)

In [170]:
datafram_tokens.tail(40)

,id,start,end,token,label,token_shift_-1,token_shift_-2,token_shift_1,token_shift_2
12,12,87,89,04,O,-,25661638,-,19
13,13,89,90,-,O,25661638,เวลาท์ฟิมฟ์,04,-
14,14,90,98,25661638,O,เวลาท์ฟิมฟ์,19,-,04
15,15,99,110,เวลาท์ฟิมฟ์,O,19,-,25661638,-
16,16,111,113,19,O,-,04,เวลาท์ฟิมฟ์,25661638
17,17,113,114,-,O,04,-,19,เวลาท์ฟิมฟ์
18,18,114,116,04,O,-,25661707,-,19
19,19,116,117,-,O,25661707,สินค้า,04,-
20,20,117,125,25661707,O,สินค้า,oty,-,04
21,21,126,132,สินค้า,O,oty,ราคา,25661707,-


In [171]:
datafram_tokens.loc[datafram_tokens['token'].fillna('').str.contains(r'[\u0E00-\u0E7Fa-zA-Z]'),'label'] = 'O'
token_filter_cost = datafram_tokens['token'].fillna('').str.contains(r'(\d+[\,\.]\d+[\,\.]\d+)|(\d+[\,\.]\d+)')
token_filter_vat = datafram_tokens['token'].str.contains(r'^(tax|vat|ภาษี|t4x7)')
token_filter_discount = datafram_tokens['token'].str.contains(r'^(ส่วนลด|discount|ลด)')
token_filter_total = datafram_tokens['token'].str.contains(r'^(รวม|total|totaไ|grandtotal|crandtota|totaไ|รวมทั้งสิ้น|รวมสุทธิ|ฟั้ง|fatal|iitai|tota1|ยิดราม|ทังทมด)|สุทธํ$|สุทธิ$|ทังสิน$|รวม$|ราม$')
token_filter_service = datafram_tokens['token'].str.contains(r'^(service|บริการ|ค่าบริการ)|charge$')
# if token_shift_1 or	token_shift_2 ^(\u0E00-\u0E7Fa-zA-Z) and (datafram_tokens['label'] == 'O') = token_filter_item
token_filter_item = datafram_tokens['token'].fillna('').str.contains(r'[\u0E00-\u0E7Fa-zA-Z]{5,30}')
token_filter_item = datafram_tokens['token_shift_1'].fillna('').str.contains(r'[\u0E00-\u0E7Fa-zA-Z]{5,30}')
token_filter_item = datafram_tokens['token_shift_2'].fillna('').str.contains(r'[\u0E00-\u0E7Fa-zA-Z]{5,30}')

# else condition 
token_filter_else = datafram_tokens['token'].str.contains(r'^(เงิ็น|เง็น|เงิน|เง็นทอน|เงิ็นสด)|included$|ทอน$|การ$')

# Filter for 'token_shift_1' column containing a number with a decimal point, handling None values
decimal_filter = datafram_tokens['token_shift_-1'].fillna('').str.contains(r'(\d+[\,\.]\d+[\,\.]\d+)|(\d+[\,\.]\d+)')
decimal_filter_spc = datafram_tokens['token_shift_-2'].fillna('').str.contains(r'(\d+[\,\.]\d+[\,\.]\d+)|(\d+[\,\.]\d+)')

# Combine the filters and update the 'label' column on the next row
datafram_tokens.loc[token_filter_cost, 'label'] = 'B-COST'
datafram_tokens.loc[(token_filter_total & (decimal_filter|decimal_filter_spc)).shift(fill_value=False), 'label'] = 'B-TOTAL'
datafram_tokens.loc[(token_filter_vat & decimal_filter).shift(fill_value=False), 'label'] = 'B-VAT'
datafram_tokens.loc[(token_filter_discount & decimal_filter).shift(fill_value=False), 'label'] = 'B-DISC'
datafram_tokens.loc[(token_filter_service & decimal_filter).shift(fill_value=False), 'label'] = 'B-SERVC'
datafram_tokens.loc[token_filter_item & (decimal_filter|decimal_filter_spc), 'label'] = 'B-ITEM'

datafram_tokens.loc[token_filter_vat | token_filter_discount | token_filter_total | token_filter_service | token_filter_else, 'label'] = 'O'
datafram_tokens.loc[(token_filter_else).shift(fill_value=False),'label'] = 'O'
datafram_tokens.loc[(token_filter_cost & decimal_filter),'label'] = 'O'

In [172]:
datafram_tokens.tail(45)

,id,start,end,token,label,token_shift_-1,token_shift_-2,token_shift_1,token_shift_2
7,7,57,67,ช้อพนักงาน,O,คุณปรม,เวลาเป้า,โต๊ะ-15,ทานฟีร้น
8,8,68,74,คุณปรม,O,เวลาเป้า,19,ช้อพนักงาน,โต๊ะ-15
9,9,75,83,เวลาเป้า,O,19,-,คุณปรม,ช้อพนักงาน
10,10,84,86,19,O,-,04,เวลาเป้า,คุณปรม
11,11,86,87,-,O,04,-,19,เวลาเป้า
12,12,87,89,04,O,-,25661638,-,19
13,13,89,90,-,O,25661638,เวลาท์ฟิมฟ์,04,-
14,14,90,98,25661638,O,เวลาท์ฟิมฟ์,19,-,04
15,15,99,110,เวลาท์ฟิมฟ์,O,19,-,25661638,-
16,16,111,113,19,O,-,04,เวลาท์ฟิมฟ์,25661638


In [173]:
dataframe_info = pd.merge(df_clean,datafram_tokens[['start','token','label']],how='inner',on='start')

### Bounding Box

In [174]:
bb_df = dataframe_info.query("label != 'O' ")
# img = image.copy()

# for x,y,w,h,label in bb_df[['left','top','width','height','label']].values:
#     x = int(x)
#     y = int(y)
#     w = int(w)
#     h = int(h)

#     cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
#     cv2.putText(img,str(label),(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)


# cv2.imshow('Predictions',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [175]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])
bb_df.head()

,text,end,start,token,label
17,1.ทงคดฝึราเมนxl,165,150,1.ทงคดฝึราเมนxl,ITEM
20,"139,00",181,175,"139,00",COST
21,2.ฟันตันเมนxl,195,182,2.ฟันตันเมนxl,ITEM
24,298.00,211,205,298.00,COST
26,ชาจีนrefill,225,214,ชาจีนrefill,ITEM


In [176]:
# group the label
class groupgen():
    def __init__(self):
        self.id = 0
        self.text = ''

    def getgroup(self,text):
        if self.text == text:
            return self.id
        else:
            self.id +=1
            self.text = text
            return self.id

grp_gen = groupgen()

In [177]:
bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)

In [178]:
# # right and bottom of bounding box
# bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
# bb_df['right'] = bb_df['left'] + bb_df['width']
# bb_df['bottom'] = bb_df['top'] + bb_df['height']

In [179]:
# # tagging: groupby group
# col_group = ['left','top','right','bottom','label','token','group']
# group_tag_img = bb_df[col_group].groupby(by='group')

In [180]:
# img_tagging = group_tag_img.agg({

#     'left':min,
#     'right':max,
#     'top':min,
#     'bottom':max,
#     'label':np.unique,
#     'token':lambda x: " ".join(x)

# })

In [181]:
# img_tagging

In [182]:
# img_tagging.info()

In [183]:
# img_bb = image.copy()
# for l,r,t,b,label,token in img_tagging.values:
#     cv2.rectangle(img_bb,(l,t),(r,b),(0,255,0),2)

#     cv2.putText(img_bb,str(label),(l,t),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)


# # cv2.imshow('Bounding Box BusinessCard',img_bb)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()
# plt.imshow(img_bb)
# plt.show()

## Parser

In [184]:
def parser(text,label):
    if label == 'ITEM':
        text = text.lower()
        text = re.sub(r'\D','',text)

    elif label == 'COST':
        text = text.lower()
        allow_special_char = '@_.\-'
        text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char),'',text)

    elif label == 'TOTAL':
        text = text.lower()
        allow_special_char = ':/.%#\-'
        text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char),'',text)

    elif label in ('VAT', 'SERVC'):
        text = text.lower()
        text = re.sub(r'[^a-z ]','',text)
        text = text.title()

    elif label == 'DISC':
        text = text.lower()
        text = re.sub(r'[^a-z0-9 ]','',text)
        text = text.title()

    return text

#### Entities

In [185]:
info_array = dataframe_info[['token','label']].values
entities = dict(ITEM=[],COST=[],TOTAL=[],VAT=[],SERVC=[],DISC=[])
previous = 'O'

for token, label in info_array:
    bio_tag = label[0]
    label_tag = label[2:]

    # step -1 parse the token
    text = parser(token,label_tag)

    if bio_tag in ('B','I'):

        if previous != label_tag:
            entities[label_tag].append(text)

        else:
            if bio_tag == "B":
                entities[label_tag].append(text)

            else:
                if label_tag in ("ITEM",'COST','DISC','VAT','TOTAL','SERVC'):
                    entities[label_tag][-1] = entities[label_tag][-1] + " " + text

                else:
                    entities[label_tag][-1] = entities[label_tag][-1] + text



    previous = label_tag


In [186]:
entities

{'ITEM': ['1', '2', '', '', ''],
 'COST': ['13900', '298.00', '60.00', '2000'],
 'TOTAL': ['517.00', '3491.15'],
 'VAT': [],
 'SERVC': [],
 'DISC': ['2585']}